# Introduction
How to open and understand the dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
def cost_derivative(a,y):
    return (a-y)/(a*(1-a))

## Basic information
1. Hyperspectral data:
    1. `hsi_path` contains path to hyperspectral masked numpy arrays containing hyperspectral data that underwent masking (i.e., the field area is masked, whereas all irrelevant areas are not masked)
    2. The name of the file (e.g., _'1989.npz'_) refers to the index of the corresponding training sample in the ground-truth table.
2. Ground-truth data:
    1. `gt_path` contains path to ground truth .csv file.
    2. Additionally, `wavelength_path` contains the mapping between a band number and the corresponding wavelength.


In [ ]:
hsi_path = 'Desktop/train_data/train_data/1331.npz'
gt_path = 'Desktop/train_data/train_gt.csv'
wavelength_path = 'Desktop/train_data/wavelengths.csv'

In [ ]:
gt_df = pd.read_csv(gt_path)
gt_df
wavelength_df = pd.read_csv(wavelength_path)
wavelength_df.head()

## Ground-truth description
`gt_df` contains:

1. `sample_index` - a reference to the numpay array containing the corresponding hyperspectral data.
2. P (for simplicity, we use P while referring to P_2O_5), K, Mg, pH - soil properties levels based on laboratory measurements.

In [ ]:
gt_df[gt_df['sample_index']==1331]

## Displaying one hyperspectral band

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
band_id = 100
wavelength = wavelength_df.loc[band_id-1]
with np.load(hsi_path) as npz:
    arr = np.ma.MaskedArray(**npz)

axs[0].imshow(arr[band_id].data)
axs[1].imshow(arr[band_id])
plt.suptitle(f'Representation of band {int(wavelength["band_no"])} ({wavelength["wavelength"]} nm)', fontsize=15)
plt.show()

## Displaying the aggregated spectral curve for a field

In [ ]:
fig = plt.figure(figsize=(10, 5))

masked_scene_mean_spectral_reflectance = [arr[i].mean() for i in range(arr.shape[0])]
full_scene_mean_spectral_reflectance = [arr[i].data.mean() for i in range(arr.shape[0])]

plt.plot(wavelength_df['wavelength'], full_scene_mean_spectral_reflectance, label='Full image')
plt.plot(wavelength_df['wavelength'], masked_scene_mean_spectral_reflectance, label='Masked image')

plt.xlabel('[nm]')
plt.legend()
plt.title(f'Average reflectance ({hsi_path.split("/")[-1]})')
plt.grid()
plt.show()

# Generating baseline solution

In [ ]:
class TestingNet:

    def __init__(self,sizes:list or np.array):
        self.num_layers=len(sizes)
        self.sizes=sizes
        self.weights=[np.random.randn(y,x)/np.sqrt(x) for x,y in zip(sizes[:-1],sizes[1:])]
        self.biases=[np.random.randn(y,1) for y in sizes[1:]]
    
    def feedforward(self,a:np.array or list):
        for w,b in zip(self.weights,self.biases):
            a=sigmoid(np.dot(w,a) + b)
        return a
    
    def LayerByLayerFF(self,X):
        Zs=[]
        As=[X]
        a=X
        for w,b in zip(self.weights,self.biases):
            z=np.dot(w,a) + b
            Zs.append(z)
            a=sigmoid(z)
            As.append(a)
        return Zs,As
    
    def update_wb(self,X,Y,eta):  #X and Y are matrices
        nabla_w,nabla_b=self.backprop(X,Y)
        self.weights=[w-(eta/X.shape[1])*dw for w,dw in zip(self.weights,nabla_w)]
        self.biases=[b-(eta/X.shape[1])*db for b,db in zip(self.biases,nabla_b)]
    
    def backprop(self,X,Y):
        Z,A=self.LayerByLayerFF(X)  # A collection of matrices.Each matrix corresponds to a layer of z(s) and a(s) for all inputs X.
        del_Ls=cost_derivative(A[-1],Y)*sigmoid_prime(Z[-1])
        deltas=[del_Ls]
        for i,(w,z) in enumerate(zip(self.weights[-1:0:-1],Z[-2::-1])):
            del_prev=np.dot(w.transpose(),deltas[i])*sigmoid_prime(z)
            deltas.append(del_prev)
        #Note:deltas contains the "errors" of each layer for all training examples (simultaneously) in reverse.
        #i.e deltas[0] has the "errors" of the last layer
        do_w=[]
        do_b=[]
        for delta_out,a_in in zip(deltas[-1::-1],A[:-1]):
            d_b=np.sum(delta_out,axis=1).reshape((len(delta_out),1))
            d_w=np.dot(delta_out,a_in.transpose()) 
            do_b.append(d_b)
            do_w.append(d_w)        
        return do_w,do_b                  
    
    def predict(self, X_test: np.ndarray):
        #X_test=[150x252] x=150x1 ====> y=4x1=>inv_sigmoid(y)
        X_test=X_test.transpose()
        predictions=self.feedforward(X_test)             
        return predictions.transpose()
        

class BaselineRegressor:
    """
    Baseline regressor, which calculates the mean value of the target from the training
    data and returns it for each testing sample.
    """
    def __init__(self):
        self.mean = 0

    def fit(self, X_train: np.ndarray, y_train: np.ndarray):
        self.mean = np.mean(y_train, axis=0)
        self.classes_count = y_train.shape[1]  #len=4
        return self

    def predict(self, X_test: np.ndarray):
        return np.full((len(X_test), self.classes_count), self.mean)


class SpectralCurveFiltering():
    """
    Create a histogram (a spectral curve) of a 3D cube, using the merge_function
    to aggregate all pixels within one band. The return array will have
    the shape of [CHANNELS_COUNT]
    """

    def __init__(self, merge_function = np.mean):
        self.merge_function = merge_function

    def __call__(self, sample: np.ndarray):
        return self.merge_function(sample[58:74], axis=(1, 2))


## Load the data

In [ ]:
import os
from glob import glob

def load_data(directory: str):
    """Load each cube, reduce its dimensionality and append to array.

    Args:
        directory (str): Directory to either train or test set
    Returns:
        [type]: A list with spectral curve for each sample.
    """
    data = []
    filtering = SpectralCurveFiltering()
    all_files = np.array(
        sorted(
            glob(os.path.join(directory, "*.npz")),
            key=lambda x: int(os.path.basename(x).replace(".npz", "")),
        )
    )
    for file_name in all_files:
        with np.load(file_name) as npz:
            arr = np.ma.MaskedArray(**npz)
        arr = filtering(arr)
        data.append(arr/100)
    return np.array(data)


def load_gt(file_path: str):
    """Load labels for train set from the ground truth file.
    Args:
        file_path (str): Path to the ground truth .csv file.
    Returns:
        [type]: 2D numpy array with soil properties levels
    """
    gt_file = pd.read_csv(file_path)
    labels = gt_file[["P", "K", "Mg", "pH"]].values
    return labels


X_train = load_data("train_data")
y_train = load_gt("train_gt.csv")
X_test = load_data("test_data")
print(X_train.shape)
#print(y_train)
print(f"Train data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


## Make predictions and generate submission file

In [ ]:
baseline_reg = BaselineRegressor()
baseline_reg = baseline_reg.fit(X_train, y_train)
predictions = baseline_reg.predict(X_test)

submission = pd.DataFrame(data = predictions, columns=["P", "K", "Mg", "pH"])
submission.to_csv("submission.csv", index_label="sample_index")


## Calculating the metric

For the purpose of presenting the final metric calculation, we will extract a small _test_set_ from the training set.

In [ ]:
X_test = X_train[1500:]
#print(X_test.shape)
y_test = y_train[1500:]
X_train_new = X_train[:]
y_train_new = y_train[:]

# Fit the baseline regressor once again on new training set

baseline_reg = baseline_reg.fit(X_train_new, y_train_new)
baseline_predictions = baseline_reg.predict(X_test)

# Generate baseline values to be used in score computation
baselines = np.mean((y_test - baseline_predictions) ** 2, axis=0)

diff_P=max(y_train_new[:,0]) - min(y_train_new[:,0])
min_P=min(y_train_new[:,0])

diff_K=max(y_train_new[:,1]) - min(y_train_new[:,1])
min_K=min(y_train_new[:,1])

diff_M=max(y_train_new[:,2]) - min(y_train_new[:,2])
min_M=min(y_train_new[:,2])

diff_p=max(y_train_new[:,3]) - min(y_train_new[:,3])
min_p=min(y_train_new[:,3])

y_train_new_2=np.zeros(y_train_new.transpose().shape)
i=0
for y,Diff,Min in zip(y_train_new.transpose(),[diff_P,diff_K,diff_M,diff_p],[min_P,min_K,min_M,min_p]):
    y_train_new_2[i]=(y-Min)/Diff
    i+=1
# Generate random predictions, different from baseline predictions
mini_batch_size=433
test=TestingNet([16,30,4])
#SGD training:
X_test.transpose())
i=0
predictions2=np.zeros(predictions.shape)
for p,Diff,Min in zip(predictions,[diff_P,diff_K,diff_M,diff_p],[min_P,min_K,min_M,min_p]):
    predictions2[i]=p*(Diff) + Min
    i+=1
predictions=predictions2.transpose()
# Calculate MSE for each class
mse = np.mean((y_test - predictions) ** 2, axis=0)
# Calculate the score for each class individually
scores = mse / baselines

# Calculate the final score
final_score = np.mean(scores)

for score, class_name in zip(scores, ["P", "K", "Mg", "pH"]):
    print(f"Class {class_name} score: {score}")

print(f"Final score: {final_scfor epoch in range(30):
    mini_batches=np.array([X_train_new.transpose()[:,k:k+mini_batch_size] for k in range(0,len(X_train_new),mini_batch_size)])
    mini_batches_out=np.array([y_train_new_2[:,k:k+mini_batch_size] for k in range(0,len(X_train_new),mini_batch_size)])
    for mini_batch,out in zip(mini_batches,mini_batches_out):
        test.update_wb(mini_batch,out,0.5)

    
predictions=test.feedforward(ore}")

In [ ]:
X_test = load_data("test_data")

In [ ]:
predictions=test.feedforward(X_test.transpose())
i=0
predictions2=np.zeros(predictions.shape)
for p,Diff,Min in zip(predictions,[diff_P,diff_K,diff_M,diff_p],[min_P,min_K,min_M,min_p]):
    predictions2[i]=p*(Diff) + Min
    i+=1
predictions=predictions2.transpose()


In [ ]:
submission = pd.DataFrame(data = predictions, columns=["P", "K", "Mg", "pH"])
submission.to_csv("submission.csv", index_label="sample_index")